# Day 13
Today I'm going to do something a little different. Instead of doing some sort of analysis I'm going to create a script to scrape [Fantasy Football Tiers](http://www.borischen.co) created by Boris Chen. All credit to him for providing this free data every week.

I'll then create some code to connect to your Yahoo Fntasy Football league so you can see at a glance who you should be starting based on the Boris Chen rankings.

## Set Up

In [2]:
from bs4 import BeautifulSoup as Soup
import pandas as pd
import requests

In [15]:
# Scrape
url = f"http://www.borischen.co/p/half-05-5-ppr-running-back-tier-rankings.html"
response = requests.get(url)

# Parse
soup = Soup(requests.get(url).content, "html.parser")

In [25]:
# Look for where the table is stored
soup.find("object")

<object data="https://s3-us-west-1.amazonaws.com/fftiers/out/text_RB-HALF.txt" style="height: 100%; margin: 1%; width: 100%;" type="text/html"></object>

In [29]:
# Get url from data tag
url = soup.find("object")['data']
url

'https://s3-us-west-1.amazonaws.com/fftiers/out/text_RB-HALF.txt'

## Extracting the Data

In [63]:
def get_player_tiers(position, scoring):

    """
    position: 'RB', 'WR', 'QB', or 'FLX
    scoring: 
        "STAN": standard
        "HALF": half-ppr
        "PPR: ppr
    """

    # Build URL
    if position == "QB":
        url = f"https://s3-us-west-1.amazonaws.com/fftiers/out/text_{position}.txt"
    else:
        if scoring == "STAN":
            url = f"https://s3-us-west-1.amazonaws.com/fftiers/out/text_{position}.txt"
        else:   
            url = f"https://s3-us-west-1.amazonaws.com/fftiers/out/text_{position}-{scoring}.txt"
    
    # Get table with player info
    table = requests.get(url).text

    # Clean up
    temp = [x.strip() for x in table.replace("\n",",").split(",")]

    # Get data into containers for saving into a DataFrame
    data = {}
    player_names = []
    tiers = []

    current_tier = 1

    for i in temp[:-1]:
        if i[:4] == "Tier":
            
            current_tier = i.split(":")[0].split(" ")[1]

            player_names.append(i.split(":")[1])
            tiers.append(current_tier)
        else:
            player_names.append(i)
            tiers.append(current_tier)

    data['player_name'] = player_names
    data['position'] = [position for i in list(range(1,len(player_names)+1))]
    data['scoring'] = [scoring for i in list(range(1,len(player_names)+1))]
    data['tier'] = tiers
    

    return pd.DataFrame(data)

# def combine_data():

In [55]:
get_player_tiers('RB', 'HALF')

,player_name,position,scoring,tier
0,Austin Ekeler,RB,HALF,1
1,Derrick Henry,RB,HALF,1
2,Alvin Kamara,RB,HALF,1
3,Josh Jacobs,RB,HALF,1
4,Aaron Jones,RB,HALF,1
5,Travis Etienne Jr.,RB,HALF,2
6,Kenneth Walker III,RB,HALF,2
7,Dalvin Cook,RB,HALF,2
8,Joe Mixon,RB,HALF,2
9,Rhamondre Stevenson,RB,HALF,2


In [56]:
get_player_tiers('QB', 'STAN')

,player_name,position,scoring,tier
0,Josh Allen,QB,STAN,1
1,Jalen Hurts,QB,STAN,1
2,Patrick Mahomes II,QB,STAN,1
3,Lamar Jackson,QB,STAN,1
4,Kyler Murray,QB,STAN,1
5,Justin Herbert,QB,STAN,1
6,Tua Tagovailoa,QB,STAN,2
7,Joe Burrow,QB,STAN,2
8,Geno Smith,QB,STAN,3
9,Justin Fields,QB,STAN,3


Imagine you are playing .5 PPR and want to know where your QBs, RBs, WRs, and FLEX players stand. You can do the following:

In [64]:
positions = ['QB', 'RB', 'WR', 'FLX']

all_players = pd.concat([
    get_player_tiers('QB', 'HALF'),
    get_player_tiers('RB', 'HALF'),
    get_player_tiers('WR', 'HALF'),
    get_player_tiers('FLX', 'HALF'),
])

,player_name,position,scoring,tier
0,Josh Allen,QB,HALF,1
1,Jalen Hurts,QB,HALF,1
2,Patrick Mahomes II,QB,HALF,1
3,Lamar Jackson,QB,HALF,1
4,Kyler Murray,QB,HALF,1
...,...,...,...,...
71,Parris Campbell,FLX,HALF,15
72,DeAndre Carter,FLX,HALF,15
73,K.J. Osborn,FLX,HALF,15
74,Isiah Pacheco,FLX,HALF,15


In [66]:
get_player_tiers('FLX', 'HALF')

,player_name,position,scoring,tier
0,Miles Sanders,FLX,HALF,1
1,Jaylen Waddle,FLX,HALF,1
2,Dameon Pierce,FLX,HALF,1
3,Amon-Ra St. Brown,FLX,HALF,1
4,Mike Evans,FLX,HALF,1
...,...,...,...,...
71,Parris Campbell,FLX,HALF,15
72,DeAndre Carter,FLX,HALF,15
73,K.J. Osborn,FLX,HALF,15
74,Isiah Pacheco,FLX,HALF,15
